# 🚀 **Passo 3: Avaliando a Velocidade dos Modelos**

## **Aula 3.1: Por Que Latência é Importante?**

---

### **Tá, mas o que é latência?**

Imagina que você tá num restaurante e pede um hambúrguer. Se o garçom demora 2 minutos pra trazer, você fica feliz. Se demora 20 minutos, você fica puto! É exatamente isso que a gente vai medir aqui - quanto tempo cada modelo de IA leva pra "cozinhar" sua resposta. 😄

**Por que latência é importante?**

Em aplicações reais, velocidade é tudo! Se você tá fazendo um chatbot e ele demora 10 segundos pra responder, o usuário já foi embora. Se você tá processando milhares de documentos por dia, cada segundo conta pro bolso.

### **Métricas de Latência que Vamos Medir**

Vamos focar em uma métrica principal pra nosso caso de uso:

| Métrica | O Que Mede | Por Que Importa |
|---------|------------|-----------------|
| **Latência Total** | Tempo total do pedido até resposta completa | Experiência do usuário final |
| **P50/P90** | 50% e 90% das respostas mais rápidas | Performance consistente |
| **Desvio Padrão** | Quão variável é a velocidade | Previsibilidade |

### **O Que Esta Avaliação Produz**

#### **1. Arquivos de Log Detalhados**

Um arquivo de log é gerado automaticamente como `model_latency_benchmarking-{timestamp}.log`, rastreando todas as chamadas de API, erros e detalhes de execução. É como ter um "gravador de voo" pra cada teste!

#### **2. Arquivos CSV de Resultados**

Resultados são salvos no diretório `../outputs/` como `document_summarization_{model_id}_{timestamp}.csv`, contendo métricas chave incluindo latência total, tempo de processamento do servidor, contagem de tokens, indicadores de status da API e detalhes de configuração.

Esses resultados serão usados no Passo 5 pra criar comparações abrangentes e visualizações.

### **Diretrizes de Benchmarking**

Pra avaliação estatisticamente válida, considere estes princípios:

| Parâmetro | Descrição | Configuração do Workshop | Recomendação de Produção |
|-----------|-----------|-------------------------|-------------------------|
| `invocations_per_scenario` | Repetições por prompt | 1 (pra eficiência) | 10+ pra significância estatística |
| `experiment_counts` | Vezes de repetir todo experimento | 1 (pra workshop) | Múltiplas execuções em dias/horários diferentes |
| `num_parallel_calls` | Requisições concorrentes | 1 (pra evitar throttling) | Combinar com sua concorrência de produção |

> **⚠️ Nota Estatística**: Enquanto estamos usando parâmetros simplificados pra este workshop, avaliações de produção devem seguir práticas estatísticas mais rigorosas. O Teorema do Limite Central nos diz que com amostras suficientes (1000+), nossas métricas vão aproximar uma distribuição normal. Idealmente, você deve:
> - Coletar amostras ao longo de múltiplos dias pra considerar variações de horário
> - Incluir períodos de pico de tráfego na sua amostragem
> - Combinar sua distribuição de teste com seus padrões reais de tráfego de produção

Vamos começar nossa avaliação de latência!

---

**🖼️ Sugestão de imagem**: Um cronômetro ou gráfico de tempo de resposta

In [ ]:
# 🛠️ IMPORTANDO AS FERRAMENTAS NECESSÁRIAS
import subprocess
import sys
import boto3
import botocore
import random
import pprint
import time
import json
import argparse
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import pytz
import os
import logging
from botocore.config import Config
from botocore.exceptions import ClientError
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
from threading import Lock
from typing import List, Dict
from tqdm.notebook import tqdm
from IPython.display import display

print("✅ Todas as ferramentas importadas! Vamos medir a velocidade!")

### **Carregando Nosso Progresso**

Vamos carregar nosso dataframe de tracking do Passo 2, que contém informações sobre nosso modelo fonte e os modelos candidatos que vamos avaliar. Esse dataframe vai servir como nosso repositório central pra todas as métricas de avaliação durante o workshop.

In [ ]:
# 📊 CARREGANDO NOSSO TRACKING
evaluation_tracking_file = '../data/evaluation_tracking.csv'
evaluation_tracking = pd.read_csv(evaluation_tracking_file)
display(evaluation_tracking)

print("\n💡 Perfeito! Agora temos nosso plano de avaliação carregado.")

### **Configuração do Setup**

#### **Definindo Nossos Parâmetros de Benchmarking**

Os parâmetros chave que vamos configurar incluem:

In [ ]:
# ⚙️ CONFIGURANDO OS PARÂMETROS DE TESTE

# Parâmetros de benchmarking
BENCHMARK_CONFIG = {
    'invocations_per_scenario': 1,  # Repetições por prompt (workshop: 1, produção: 10+)
    'experiment_counts': 1,  # Vezes de repetir todo experimento (workshop: 1, produção: múltiplas)
    'num_parallel_calls': 1,  # Chamadas concorrentes (workshop: 1, produção: baseado na concorrência)
    'timeout_seconds': 300,  # Timeout pra cada chamada (5 minutos)
    'retry_attempts': 3,  # Tentativas de retry em caso de erro
    'delay_between_calls': 1.0  # Delay entre chamadas (segundos)
}

print("⚙️ CONFIGURAÇÃO DE BENCHMARKING:")
for key, value in BENCHMARK_CONFIG.items():
    print(f"• {key}: {value}")

print("\n💡 Esses são os parâmetros que vamos usar pra medir a velocidade dos modelos.")
print("💡 Em produção, você aumentaria esses números pra ter resultados mais confiáveis.")

### **Configurando o Sistema de Logging**

Vamos configurar um sistema de logging robusto pra rastrear todas as nossas chamadas de API. É como ter um "gravador de voo" que registra tudo que acontece durante os testes!

In [ ]:
# 📝 CONFIGURANDO O SISTEMA DE LOGGING
def setup_logging(model_id):
    """
    Configura o sistema de logging pra um modelo específico.
    É como configurar uma câmera de segurança que filma tudo!
    """
    
    # Criando nome do arquivo de log
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    log_filename = f'model_latency_benchmarking_{model_id.replace(":", "-")}_{timestamp}.log'
    
    # Configurando o logger
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_filename),
            logging.StreamHandler()
        ]
    )
    
    logger = logging.getLogger(__name__)
    logger.info(f"🚀 Iniciando benchmark para modelo: {model_id}")
    logger.info(f"📝 Arquivo de log: {log_filename}")
    
    return logger, log_filename

print("✅ Sistema de logging configurado! Vamos rastrear tudo que acontece.")

### **Configurando o Cliente Bedrock**

Vamos configurar o cliente Bedrock com configurações otimizadas pra benchmarking. É como ajustar o carro antes de uma corrida!

In [ ]:
# 🔧 CONFIGURANDO O CLIENTE BEDROCK
def create_bedrock_client(region='us-east-1'):
    """
    Cria um cliente Bedrock otimizado pra benchmarking.
    É como configurar um carro de corrida com as melhores configurações!
    """
    
    # Configurações otimizadas pra performance
    config = Config(
        region_name=region,
        retries={
            'max_attempts': 3,
            'mode': 'adaptive'
        },
        connect_timeout=60,
        read_timeout=300
    )
    
    client = boto3.client('bedrock-runtime', config=config)
    
    return client

print("✅ Cliente Bedrock configurado com otimizações de performance!")

### **Função Principal de Benchmarking**

Agora vamos criar a função principal que vai fazer o trabalho pesado de medir a latência. É como ter um cronômetro profissional que mede tudo com precisão!

In [ ]:
# ⏱️ FUNÇÃO PRINCIPAL DE BENCHMARKING
def benchmark_model_latency(model_id, prompt, dataset_path, logger):
    """
    Executa o benchmark de latência para um modelo específico.
    É como cronometrar cada volta de uma corrida!
    """
    
    results = []
    client = create_bedrock_client()
    
    # Carregando o dataset
    dataset = pd.read_csv(dataset_path)
    logger.info(f"�� Dataset carregado: {len(dataset)} amostras")
    
    # Loop principal de benchmarking
    for scenario_idx in range(BENCHMARK_CONFIG['experiment_counts']):
        logger.info(f"🔄 Experimento {scenario_idx + 1}/{BENCHMARK_CONFIG['experiment_counts']}")
        
        for invocation_idx in range(BENCHMARK_CONFIG['invocations_per_scenario']):
            logger.info(f"  📝 Invocação {invocation_idx + 1}/{BENCHMARK_CONFIG['invocations_per_scenario']}")
            
            # Processando cada documento no dataset
            for doc_idx, row in dataset.iterrows():
                document = row['document']
                reference_response = row['referenceResponse']
                
                # Formatando o prompt
                formatted_prompt = prompt.format(context=document)
                
                # Fazendo a chamada da API
                start_time = time.time()
                
                try:
                    # Preparando a requisição
                    request_body = {
                        'messages': [
                            {
                                'role': 'user',
                                'content': [{'text': formatted_prompt}]
                            }
                        ],
                        'inferenceConfig': {
                            'temperature': 0.7,
                            'maxTokens': 1000,
                            'topP': 0.9
                        }
                    }
                    
                    # Chamada da API
                    response = client.converse(
                        modelId=model_id,
                        body=json.dumps(request_body)
                    )
                    
                    end_time = time.time()
                    
                    # Extraindo a resposta
                    model_response = response['output']['message']['content'][0]['text']
                    
                    # Calculando métricas
                    total_latency = end_time - start_time
                    
                    # Extraindo informações de tokens (se disponível)
                    input_tokens = response.get('usage', {}).get('inputTokens', 0)
                    output_tokens = response.get('usage', {}).get('outputTokens', 0)
                    
                    # Criando resultado
                    result = {
                        'model': model_id,
                        'region': 'us-east-1',
                        'inference_profile': 'standard',
                        'document': document,
                        'referenceResponse': reference_response,
                        'model_response': model_response,
                        'latency': total_latency,
                        'model_latencyMs': total_latency * 1000,  # Convertendo pra milissegundos
                        'model_input_tokens': input_tokens,
                        'model_output_tokens': output_tokens,
                        'status': 'success',
                        'timestamp': datetime.now().isoformat()
                    }
                    
                    results.append(result)
                    
                    logger.info(f"    ✅ Documento {doc_idx + 1}: {total_latency:.3f}s")
                    
                except Exception as e:
                    end_time = time.time()
                    total_latency = end_time - start_time
                    
                    result = {
                        'model': model_id,
                        'region': 'us-east-1',
                        'inference_profile': 'standard',
                        'document': document,
                        'referenceResponse': reference_response,
                        'model_response': f"ERROR: {str(e)}",
                        'latency': total_latency,
                        'model_latencyMs': total_latency * 1000,
                        'model_input_tokens': 0,
                        'model_output_tokens': 0,
                        'status': 'error',
                        'error_message': str(e),
                        'timestamp': datetime.now().isoformat()
                    }
                    
                    results.append(result)
                    logger.error(f"    ❌ Documento {doc_idx + 1}: Erro - {str(e)}")
                
                # Delay entre chamadas
                time.sleep(BENCHMARK_CONFIG['delay_between_calls'])
    
    logger.info(f"🎉 Benchmark concluído para {model_id}!")
    logger.info(f"📊 Total de resultados: {len(results)}")
    
    return results

print("✅ Função de benchmarking criada! Vamos começar a medir!")

### **Executando os Benchmarks**

Agora vamos executar os benchmarks pra todos os nossos modelos candidatos. É como fazer uma corrida cronometrada com todos os carros!

In [ ]:
# 🏁 EXECUTANDO OS BENCHMARKS
print("�� INICIANDO BENCHMARKS DE LATÊNCIA...")
print("=" * 60)

all_results = []
dataset_path = '../data/document_sample_10.csv'  # Usando amostra pequena pro workshop

for index, row in evaluation_tracking.iterrows():
    model_id = row['model']
    prompt = row['text_prompt']
    
    # Pular se não tem prompt (modelo fonte)
    if not prompt or model_id == 'source_model':
        print(f"⏭️ Pulando {model_id} (sem prompt ou modelo fonte)")
        continue
    
    print(f"\n🎯 BENCHMARK PARA: {model_id}")
    print("-" * 40)
    
    # Configurando logging
    logger, log_filename = setup_logging(model_id)
    
    try:
        # Executando benchmark
        results = benchmark_model_latency(model_id, prompt, dataset_path, logger)
        all_results.extend(results)
        
        # Salvando resultados
        results_df = pd.DataFrame(results)
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        output_filename = f'../outputs/document_summarization_{model_id.replace(":", "-")}_{timestamp}.csv'
        results_df.to_csv(output_filename, index=False)
        
        print(f"✅ Resultados salvos em: {output_filename}")
        print(f"�� Total de amostras: {len(results)}")
        
        # Estatísticas rápidas
        successful_results = [r for r in results if r['status'] == 'success']
        if successful_results:
            latencies = [r['latency'] for r in successful_results]
            print(f"⏱️ Latência média: {np.mean(latencies):.3f}s")
            print(f"⏱️ Latência mediana: {np.median(latencies):.3f}s")
            print(f"⏱️ Latência P90: {np.percentile(latencies, 90):.3f}s")
        
    except Exception as e:
        print(f"❌ Erro no benchmark de {model_id}: {str(e)}")
        logger.error(f"Erro fatal: {str(e)}")

print("\n" + "=" * 60)
print("🎉 TODOS OS BENCHMARKS CONCLUÍDOS!")

### **Analisando os Resultados**

Agora vamos dar uma olhada nos resultados que coletamos. É como analisar os tempos de uma corrida pra ver quem foi mais rápido!

In [ ]:
# 📊 ANALISANDO OS RESULTADOS
print("�� ANÁLISE DOS RESULTADOS DE LATÊNCIA")
print("=" * 50)

# Carregando todos os resultados salvos
import glob
import os

output_directory = '../outputs'
csv_files = glob.glob(os.path.join(output_directory, 'document_summarization_*.csv'))

all_data = []
for file in csv_files:
    if 'source_model' not in file:  # Excluindo modelo fonte
        df = pd.read_csv(file)
        all_data.append(df)

if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)
    
    print(f"📊 Total de amostras coletadas: {len(combined_df)}")
    print(f"🎯 Modelos testados: {combined_df['model'].unique()}")
    
    # Estatísticas por modelo
    print("\n📈 ESTATÍSTICAS POR MODELO:")
    print("-" * 40)
    
    for model in combined_df['model'].unique():
        model_data = combined_df[combined_df['model'] == model]
        successful_data = model_data[model_data['status'] == 'success']
        
        if len(successful_data) > 0:
            latencies = successful_data['latency']
            
            print(f"\n🎯 {model}:")
            print(f"  📊 Amostras: {len(successful_data)}")
            print(f"  ⏱️ Média: {latencies.mean():.3f}s")
            print(f"  ⏱️ Mediana: {latencies.median():.3f}s")
            print(f"  ⏱️ P90: {latencies.quantile(0.9):.3f}s")
            print(f"  ⏱️ Mín: {latencies.min():.3f}s")
            print(f"  ⏱️ Máx: {latencies.max():.3f}s")
            print(f"  📈 Desvio Padrão: {latencies.std():.3f}s")
        else:
            print(f"\n❌ {model}: Nenhum resultado bem-sucedido")

else:
    print("❌ Nenhum resultado encontrado. Verifique se os benchmarks foram executados.")

### **Visualizando os Resultados**

Vamos criar algumas visualizações pra entender melhor os resultados. É como transformar números em gráficos que contam uma história!

In [ ]:
# �� CRIANDO VISUALIZAÇÕES
if 'combined_df' in locals() and len(combined_df) > 0:
    # Filtrando apenas resultados bem-sucedidos
    successful_df = combined_df[combined_df['status'] == 'success']
    
    if len(successful_df) > 0:
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Gráfico 1: Boxplot de latência por modelo
        models = successful_df['model'].unique()
        latency_data = [successful_df[successful_df['model'] == model]['latency'] for model in models]
        
        axes[0, 0].boxplot(latency_data, labels=models)
        axes[0, 0].set_title('⏱️ Distribuição de Latência por Modelo')
        axes[0, 0].set_ylabel('Latência (segundos)')
        axes[0, 0].tick_params(axis='x', rotation=45)
        
        # Gráfico 2: Histograma de latência
        for model in models:
            model_data = successful_df[successful_df['model'] == model]['latency']
            axes[0, 1].hist(model_data, alpha=0.7, label=model, bins=10)
        
        axes[0, 1].set_title('📊 Distribuição de Latência')
        axes[0, 1].set_xlabel('Latência (segundos)')
        axes[0, 1].set_ylabel('Frequência')
        axes[0, 1].legend()
        
        # Gráfico 3: Latência vs tokens de entrada
        for model in models:
            model_data = successful_df[successful_df['model'] == model]
            axes[1, 0].scatter(model_data['model_input_tokens'], 
                              model_data['latency'], 
                              alpha=0.6, label=model)
        
        axes[1, 0].set_title('�� Latência vs Tokens de Entrada')
        axes[1, 0].set_xlabel('Tokens de Entrada')
        axes[1, 0].set_ylabel('Latência (segundos)')
        axes[1, 0].legend()
        
        # Gráfico 4: Comparação de métricas
        metrics_comparison = []
        model_names = []
        
        for model in models:
            model_data = successful_df[successful_df['model'] == model]['latency']
            metrics_comparison.append([
                model_data.mean(),
                model_data.median(),
                model_data.quantile(0.9)
            ])
            model_names.append(model.split('.')[-1])  # Nome mais curto
        
        metrics_comparison = np.array(metrics_comparison)
        x = np.arange(len(model_names))
        width = 0.25
        
        axes[1, 1].bar(x - width, metrics_comparison[:, 0], width, label='Média', alpha=0.8)
        axes[1, 1].bar(x, metrics_comparison[:, 1], width, label='Mediana', alpha=0.8)
        axes[1, 1].bar(x + width, metrics_comparison[:, 2], width, label='P90', alpha=0.8)
        
        axes[1, 1].set_title('📈 Comparação de Métricas de Latência')
        axes[1, 1].set_xlabel('Modelo')
        axes[1, 1].set_ylabel('Latência (segundos)')
        axes[1, 1].set_xticks(x)
        axes[1, 1].set_xticklabels(model_names)
        axes[1, 1].legend()
        
        plt.tight_layout()
        plt.show()
        
        print("\n💡 O que esses gráficos nos dizem?")
        print("• O boxplot mostra a distribuição de latência de cada modelo")
        print("• O histograma mostra como a latência se distribui")
        print("• O scatter plot mostra se há relação entre tamanho do input e latência")
        print("• O gráfico de barras compara as métricas principais")
    
else:
    print("❌ Não há dados suficientes para criar visualizações.")

### **Resumo do Passo 3**

 **Parabéns!** Você acabou de completar o terceiro passo da nossa jornada de migração. Vamos recapitular o que fizemos:

✅ **Entendemos a importância da latência**: Velocidade é crucial em aplicações reais
✅ **Configuramos benchmarking robusto**: Sistema completo de medição
✅ **Executamos testes de latência**: Medimos a velocidade de todos os modelos
✅ **Coletamos métricas detalhadas**: Latência, tokens, status de cada chamada
✅ **Analisamos os resultados**: Estatísticas e visualizações
✅ **Salvamos dados estruturados**: CSV com todos os resultados

### **O Que Vem no Próximo Passo**

No próximo notebook, vamos fazer algo super importante: **avaliar a qualidade**! É como provar a comida depois de cronometrar o tempo de preparo. Vamos usar o LLM-as-a-Judge do Bedrock pra avaliar automaticamente a qualidade das respostas que geramos.

---

**💡 Dica do Pedro**: Latência é só uma parte da história! Um modelo pode ser rápido, mas se a qualidade for ruim, não adianta nada. É como ter um carro rápido que quebra toda hora!

**🚀 Próximo passo**: Avaliação de qualidade com LLM-as-a-Judge